In [ ]:
#Load libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform

In [ ]:
from fancyimpute import KNN

In [ ]:
#Set working directory
os.chdir(r"C:\Users\RAUNAK\Desktop\edwisor\workspace")

In [ ]:
#Load data
bike_rental = pd.read_csv("day.csv")

## Exploratory Data Analysis

In [ ]:
#bike_rental= pd.DataFrame(bike_rental)
bike_rental.info()

In [ ]:
# Remove instant and dteday because they do not serve any purpose
# Remove casual and registered because they are what we will predict
bike_rental = bike_rental.drop(['instant','dteday', 'casual','registered'], axis=1)

In [ ]:
bike_rental

## Missing Value Analysis


In [ ]:
#Check if there are any missing values
bike_rental.isnull().sum()

In [ ]:
#Convert into proper datatypes
list = ["season", "yr","mnth","holiday","workingday","weekday","weathersit"]
for i in list:
    bike_rental.loc[:,i] = bike_rental.loc[:,i].round()
    bike_rental.loc[:,i] = bike_rental.loc[:,i].astype('object')

## Outlier Analysis

In [ ]:
df = bike_rental.copy()
#bike_rental = df.copy()

In [ ]:
# #Plot boxplot to visualize Outliers
# %matplotlib inline  
#plt.boxplot(bike_rental['temp'])
#plt.boxplot(bike_rental['atemp'])
#plt.boxplot(bike_rental['hum'])
plt.boxplot(bike_rental['windspeed'])

In [ ]:
#save numeric names
cnames =  ["temp","atemp","hum","windspeed"]

In [ ]:
#bike_rental.loc[bike_rental['hum'] < minimum,'hum'] = np.nan
# bike_rental.loc[bike_rental['hum'] > maximum,:'hum'] = np.nan

bike_rental.loc[bike_rental['hum'] > maximum,'hum'] = np.nan


In [ ]:
#bike_rental.loc[bike_rental['hum'] < minimum,'hum'] = np.nan
# bike_rental.loc[bike_rental['hum'] > maximum,:'hum'] = np.nan

#bike_rental['windspeed']<minimum

#bike_rental.loc[bike_rental['windspeed']>maximum]['windspeed'] = np.nan

bike_rental.loc[bike_rental['windspeed']>maximum,'windspeed'] =np.nan


In [ ]:
#Detect and replace with NA
# #Extract quartiles for hum
q75, q25 = np.percentile(bike_rental['windspeed'], [75 ,25])

# #Calculate IQR
#iqr = q75 - q25

# #Calculate inner and outer fence
#minimum = q25 - (iqr*1.5)

#minimum
    
    
maximum = q75 + (iqr*1.5)

# #Replace with NA
#bike_rental.loc[bike_rental['hum'] < minimum,'hum'] = np.nan
#bike_rental.loc[bike_rental['hum'] > maximum,'hum'] = np.nan



In [ ]:
# #Impute with KNN
bike_rental = pd.DataFrame(KNN(k = 3).complete(bike_rental), columns = bike_rental.columns)

In [ ]:
#Detect and replace with NA
# #Extract quartiles for hum
 q75, q25 = np.percentile(bike_rental['hum'], [75 ,25])

# #Calculate IQR
 iqr = q75 - q25

# #Calculate inner and outer fence
 minimum = q25 - (iqr*1.5)
 maximum = q75 + (iqr*1.5)

# #Replace with NA
 bike_rental.loc[bike_rental['hum'] < minimum,'hum'] = np.nan
 bike_rental.loc[bike_rental['hum'] > maximum,'hum'] = np.nan


# #Calculate missing value
 missing_val = pd.DataFrame(bike_rental.isnull().sum())

# #Impute with KNN
bike_rental = pd.DataFrame(KNN(k = 3).complete(bike_rental), columns = bike_rental.columns)

## Feature Selection

In [ ]:
##Correlation analysis
#Correlation plot
df_corr =bike_rental.loc[:,cnames]

In [ ]:
#Set the width and hieght of the plot
f, ax = plt.subplots(figsize=(7, 5))

#Generate correlation matrix
corr = df_corr.corr()


In [ ]:
#Dimension Reduction. 
#We are removing atemp because it is highly correlated to temp
bike_rental = bike_rental.drop(['atemp'], axis=1)

## Feature Scaling

In [ ]:
#df = marketing_train.copy()
#marketing_train = df.copy()

In [ ]:
#Normality check
%matplotlib inline  
plt.hist(bike_rental['hum'], bins='auto')

In [ ]:
%matplotlib inline  
plt.hist(bike_rental['windspeed'], bins='auto')

In [ ]:
#The graphs for 'hum' and 'windspeed' look normally distributed after outlier removal .

## Model Development

**Decision Tree Regression**

In [ ]:
#Import Libraries for decision tree
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor


In [ ]:
#Divide data into train and test
train, test = train_test_split(bike_rental, test_size=0.2)

In [ ]:
#Decision tree for regression
fit_DT = DecisionTreeRegressor(max_depth=2).fit(train.iloc[:,0:10], train.iloc[:,10])



In [ ]:
#Apply model on test data
predictions_DT = fit_DT.predict(test.iloc[:,0:10])




In [ ]:



#Calculate MAPE
def MAPE(y_true, y_pred): 
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    return mape





In [ ]:
#Calculate MAPE value in %
MAPE(test.iloc[:,10], predictions_DT)

**Multiple Linear Regression**

In [ ]:
#Import libraries for Linear Regression
import statsmodels.api as sm


In [ ]:
# Train the model using the training sets
model = sm.OLS(train.iloc[:,10].astype(float), train.iloc[:,0:10].astype(float)).fit()




In [ ]:
# Print out the statistics
model.summary()


In [ ]:
# make the predictions by the model
predictions_LR = model.predict(test.iloc[:,0:10]) 


In [ ]:
#Calculate MAPE
MAPE(test.iloc[:,10], predictions_LR)

**Random Forest Regression**

In [ ]:
#Import libraries for Random Forest
from sklearn.ensemble import RandomForestClassifier



In [ ]:
# Train the model using the training sets
RF_model = RandomForestClassifier(n_estimators = 20).fit(train.iloc[:,0:10], train.iloc[:,10])

In [ ]:
# make the predictions by the model
RF_Predictions = RF_model.predict(test.iloc[:,0:10])

In [ ]:
#Calculate MAPE
MAPE(test.iloc[:,10], RF_Predictions)